# ML Zoomcamp 2025 – Homework 9 (Serverless)

This notebook was generated from `homework.md` and contains code scaffolding and helper utilities
to solve the **Straight vs Curly Hair Type** homework.


## Original homework text

## Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the
[previous homework](../08-deep-learning/homework.md).

Download the model files from here: 

* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

With wget:

```bash
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL="${PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL="${PREFIX}/hair_classifier_v1.onnx"
wget ${DATA_URL}
wget ${MODEL_URL}
```


## Question 1

To be able to use this model, we need to know the name of the input and output nodes. 

What's the name of the output:

* `output`
* `sigmoid`
* `softmax`
* `prediction`


## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

## Question 2: Target size

Let's download and resize this image: 

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

* 64x64
* 128x128
* 200x200
* 256x256


## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* -10.73
* -1.073
* 1.073
* 10.73


## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.09
* 0.49
* 0.69
* 0.89

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.13

COPY hair_classifier_empty.onnx.data .
COPY hair_classifier_empty.onnx .
```

Note that it uses Python 3.13.

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.


## Question 5

Download the base image `agrigorev/model-2025-hairstyle:v1`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 88 Mb
* 208 Mb
* 608 Mb
* 1208 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `hair_classifier_empty.onnx` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* -1.0
* -0.10
* 0.10
* 1.0


## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.


## Submit the results

* Submit your results here: https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw09
* If your answer doesn't match options exactly, select the closest one. If the answer is exactly in between two options, select the higher value.

## Publishing to Docker hub

Just for the reference, this is how we published our image to Docker hub:

```bash
docker build -t model-2025-hairstyle -f homework.dockerfile .
docker tag model-2025-hairstyle:latest agrigorev/model-2025-hairstyle:v1
docker push agrigorev/model-2025-hairstyle:v1
```

(You don't need to execute this code)


## Setup

We'll need a few Python packages:

- `numpy`
- `Pillow` (PIL)
- `onnx`
- `onnxruntime`
- `requests`
- (later, for AWS parts) `boto3`

You can install anything that's missing with:

```bash
pip install numpy pillow onnx onnxruntime requests boto3
```


## Question 1 – Inspecting the ONNX model

In [28]:
# Download the ONNX model files (if you haven't already done it with wget)
import os
import requests

PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

def download(url, fname):
    if os.path.exists(fname):
        print(f"{fname} already exists")
        return
    print(f"Downloading {url} -> {fname}")
    resp = requests.get(url)
    resp.raise_for_status()
    with open(fname, "wb") as f_out:
        f_out.write(resp.content)
    print("Done")

download(DATA_URL, "hair_classifier_v1.onnx.data")
download(MODEL_URL, "hair_classifier_v1.onnx")

hair_classifier_v1.onnx.data already exists
hair_classifier_v1.onnx already exists


In [29]:
# Inspect the ONNX model: inputs and outputs
import onnx

model = onnx.load("hair_classifier_v1.onnx")

print("=== Inputs ===")
for i, inp in enumerate(model.graph.input):
    t = inp.type.tensor_type
    shape = [d.dim_value for d in t.shape.dim]
    print(f"{i}: name={inp.name}, shape={shape}")

print("\n=== Outputs ===")
for i, out in enumerate(model.graph.output):
    t = out.type.tensor_type
    shape = [d.dim_value for d in t.shape.dim]
    print(f"{i}: name={out.name}, shape={shape})")

# The name of the only output node printed above is what you need for Question 1.


=== Inputs ===
0: name=input, shape=[0, 3, 200, 200]

=== Outputs ===
0: name=output, shape=[0, 1])


## Questions 2 & 3 – Downloading and preprocessing the image

We'll download the test image from the homework and apply the **same preprocessing**
we used in module 8 (deep learning homework).

> ⚠️ IMPORTANT: adjust `TARGET_SIZE` and the preprocessing block to match whatever
> you actually used previously (e.g. scaling to `[0, 1]`, or to `[-1, 1]`, or using
> a Keras `preprocess_input` function).


In [30]:
# Download the test image
import requests
from pathlib import Path

IMAGE_URL = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
IMAGE_PATH = Path("test-image.jpeg")

if not IMAGE_PATH.exists():
    print(f"Downloading test image from {IMAGE_URL}")
    resp = requests.get(IMAGE_URL)
    resp.raise_for_status()
    IMAGE_PATH.write_bytes(resp.content)
    print("Saved to", IMAGE_PATH)
else:
    print("Image already downloaded:", IMAGE_PATH)

Image already downloaded: test-image.jpeg


In [31]:
# Resize and preprocess the image
import numpy as np
from PIL import Image

# TODO: set this to the value you used in module 8.
# In many solutions, the model was trained on 200x200 images.
TARGET_SIZE = (200, 200)

def load_and_preprocess(path: str, target_size=TARGET_SIZE):
    img = Image.open(path).convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    x = np.array(img).astype("float32")

    # ----- PREPROCESSING (adapt to what you actually used) -----
    # Same normalization as in homework 8 (ImageNet-style)
    x = x / 255.0
    mean = np.array([0.485, 0.456, 0.406], dtype="float32")
    std = np.array([0.229, 0.224, 0.225], dtype="float32")
    x = (x - mean) / std
    # -----------------------------------------------------------

    return x

x = load_and_preprocess(str(IMAGE_PATH))

print("Image array shape (H, W, C):", x.shape)
print("First pixel (R, G, B):", x[0, 0])

# For Question 3, you'll need x[0, 0, 0] – the R channel of the first pixel
r_first_pixel = x[0, 0, 0]
print("First pixel R channel:", r_first_pixel)

Image array shape (H, W, C): (200, 200, 3)
First pixel (R, G, B): [-1.073294   -0.21498597 -1.4210021 ]
First pixel R channel: -1.073294


## Question 4 – Running the ONNX model locally

Now we'll run the ONNX model with **onnxruntime** on the preprocessed image and
look at the raw output.


In [32]:
import numpy as np
import onnxruntime as ort

# Create a session
session = ort.InferenceSession("hair_classifier_v1.onnx", providers=["CPUExecutionProvider"])

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

print("Input name:", input_name)
print("Output name:", output_name)

# ONNX models exported from Keras usually expect NCHW: (1, 3, H, W)
# Our x is HWC, so we need to transpose.
x_for_onnx = np.expand_dims(np.transpose(x, (2, 0, 1)), axis=0)  # (1, 3, H, W)

print("Input tensor shape:", x_for_onnx.shape)

pred = session.run([output_name], {input_name: x_for_onnx})[0]
print("Raw model output:", pred)

# For the multiple-choice question, inspect pred (e.g. a single logit or a 2-element vector)


Input name: input
Output name: output
Input tensor shape: (1, 3, 200, 200)
Raw model output: [[0.09156588]]


## Question 5 – Packaging as a Docker container

Follow the homework instructions to:

1. Create a `predict.py` script that:
   - Loads the ONNX model once at import time.
   - Exposes a `lambda_handler(event, context)` (or similar) that:
     - Parses the incoming event (base64-encoded image or URL, depending on your design),
     - Downloads + preprocesses the image (using `load_and_preprocess`),
     - Runs the ONNX model,
     - Returns the prediction.
2. Build a Docker image locally (you can base it off `public.ecr.aws/lambda/python:3.12` or the image from the homework).
3. Test the container locally with `docker run` and `curl`.

You don't strictly need a notebook for this part; keeping the code in `predict.py`
and a `Dockerfile` (or `homework.dockerfile`) is usually more convenient.


## Question 6 – Deploying to AWS Lambda

For this part, you'll:

1. Push your image to **Amazon ECR**.
2. Create a **Lambda function** using **Container image** as the source.
3. Increase memory and timeout a bit (e.g. 1 GB RAM and 30–60 seconds).
4. Test it from the AWS Console.
5. (Optional but recommended) Expose it via **API Gateway** and test from your local machine.

Because this part is mostly AWS console work, there isn't a lot of additional
Jupyter code to write here. You can still use cells in this notebook to experiment
with calling your public endpoint once it's ready.


In [33]:
# OPTIONAL: Example of calling your Lambda via API Gateway once it's deployed
# Replace the URL below with your own API Gateway invoke URL.
import requests
import json

GATEWAY_URL = "https://your-api-id.execute-api.your-region.amazonaws.com/prod/predict"

# Example payload – you need to match whatever your lambda expects,
# e.g. it could be an image URL or a base64-encoded image.
payload = {
    "image_url": str(IMAGE_PATH.resolve().as_posix())
}

# Uncomment when you have your endpoint ready:
# resp = requests.post(GATEWAY_URL, json=payload)
# print(resp.status_code, resp.text)